## Подготовка данных


Выбранный текст - Человек, который принял жену за шляпу

In [96]:
#constants 
file_name = 'Человек,который_принял_жену_за_шляпу.pdf' # реальный текст романа с 14 по 112 страницы 
sentence_len = 50

In [97]:
! pip install pdfquery
! pip install pandas
! pip install --user -U nltk

import sys
!{sys.executable} -m pip install numpy

! pip install torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new 

In [98]:
#imports 
from pdfquery import PDFQuery

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string
import re

from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm

In [99]:
pdf = PDFQuery(file_name)
pdf.load(*range(13, 50)) #TODO("увеличить?")

text_elements = pdf.pq('LTTextLineHorizontal')

In [100]:
text = "".join([t.text for t in text_elements]).replace('\n', '')

In [101]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/esoboleva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [102]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting other symbols
    punct = ['—', ',', '.', '...']
    tokens = [i for i in tokens if (i not in punct)]

    # cleaning words
    tokens = [re.sub("[a-z]","", re.sub("[0-9]","",
              i.replace("«", "")
              .replace("»", "")
              .replace(".", "")
              .replace("-", "")
              .replace('—', "")
              .replace(',', "")
              .replace('`', "")
                                        
              .lower()))
              for i in tokens]
    

    return tokens

In [103]:
sentences = [tokenize_ru(sent) for sent in sent_tokenize(text, 'russian')]

In [104]:
alphabet = {'\n'}

loaded_text_path = 'text.txt'
text_file = open(loaded_text_path, "w")

for s in sentences:
    joined_s = " ".join(s)
    if len(joined_s) > sentence_len:
        text_file.write(joined_s)
        text_file.write('\n')
        # getting alphabet
        for c in joined_s:
            alphabet.add(c)
text_file.close()
alphabet = sorted(alphabet)
print(alphabet)
n_vocab = len(alphabet)

['\n', ' ', "'", 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']


остались только кириллица и пробел - ура

сделаем one-hot encoding

In [105]:
text = open("text.txt").read()
sentences = open("text.txt").readlines()
print('Symbols count: ', len(text))
print('Sentences count: ', len(sentences))

Symbols count:  158422
Sentences count:  1204


In [106]:
def encode_symbol(symbol: str, encoder):
    encoded_symbol = [0] * len(encoder)
    encoded_symbol[encoder[symbol]] = 1
    return encoded_symbol


def encode_string(string: str, encoder):
    encoded_string = []
    for symbol in string:
        encoded_string.append(encode_symbol(symbol, encoder))
    return encoded_string


def encode_data(data):
    symbols = list(set([symbol for string in data for symbol in string]))
    count_symbols = len(symbols)
    encoder = dict(zip(symbols, range(count_symbols)))
    encoded_data = []
    for string in tqdm(data, desc="Encode sentences"):
        encoded_data.append(encode_string(string, encoder))
    return encoded_data, encoder, {v: k for k, v in encoder.items()}

encoded_sentences, encoder_map, decoder_map = encode_data(sentences)

def get_data(sentences, length=sentence_len):
    xs = []
    ys = []
    for sentence in tqdm(sentences, desc="Cut sentences"):
        if len(sentence) <= length:
            continue
        for i in range(0, len(sentence) - length):
            x = sentence[i:i + length]
            y = sentence[i + length]
            xs.append(x)
            ys.append(y)
    return np.array(xs), np.array(ys)

data_x, data_y = get_data(encoded_sentences)
print("Shapes: ", data_x.shape, " ", data_y.shape)

X = torch.tensor(data_x, dtype=torch.float32)
y = torch.tensor(data_y, dtype=torch.float32)

Cut sentences: 100%|█████████████████████| 1204/1204 [00:00<00:00, 22809.44it/s]


Shapes:  (98222, 50, 35)   (98222, 35)


## LSTM

In [107]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_vocab, hidden_size=256, num_layers=3, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(256, n_vocab)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(x)
        return x

In [108]:
n_epochs = 30
batch_size = 128
model = RNNModel()

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
loader = DataLoader(TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss / batch_size))

Epoch 0: Cross-entropy: 14.2383
Epoch 1: Cross-entropy: 12.0748
Epoch 2: Cross-entropy: 10.4981
Epoch 3: Cross-entropy: 9.4875
Epoch 4: Cross-entropy: 8.7264
Epoch 5: Cross-entropy: 8.1136
Epoch 6: Cross-entropy: 7.6004
Epoch 7: Cross-entropy: 7.1199
Epoch 8: Cross-entropy: 6.5495
Epoch 9: Cross-entropy: 6.1052
Epoch 10: Cross-entropy: 5.6048
Epoch 11: Cross-entropy: 5.1701
Epoch 12: Cross-entropy: 4.6408
Epoch 13: Cross-entropy: 4.2214
Epoch 14: Cross-entropy: 3.7302
Epoch 15: Cross-entropy: 3.3445
Epoch 16: Cross-entropy: 2.9119
Epoch 17: Cross-entropy: 2.5427
Epoch 18: Cross-entropy: 2.2303
Epoch 19: Cross-entropy: 1.9532
Epoch 20: Cross-entropy: 1.6983
Epoch 21: Cross-entropy: 1.4667
Epoch 22: Cross-entropy: 1.3687
Epoch 23: Cross-entropy: 1.1515
Epoch 24: Cross-entropy: 1.1068
Epoch 25: Cross-entropy: 1.1054
Epoch 26: Cross-entropy: 0.9395
Epoch 27: Cross-entropy: 0.9158
Epoch 28: Cross-entropy: 0.8161
Epoch 29: Cross-entropy: 0.8358


In [113]:
torch.save([best_model, char_to_int], "model.pth")

In [114]:
file = open('text.txt', 'r')
lines = file.readlines()

In [ ]:
prompt_len = 15
sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

In [ ]:
pattern = [encoder_map[c] for c in prompt]

model.eval()
print("Real sentence:\n", lines[sentence])
print('Prompt: "%s"' % prompt)
print(prompt, end="")
with torch.no_grad():
    for i in range(500):
        # format input array of int into PyTorch tensor
#         x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(X, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(prompt)
        # convert logits into one character
        index = int(prediction.argmax())
        result = decoder_map[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

## Марковская цепь


In [ ]:
n_chars_to_consider = 10
transitions = dict()

In [ ]:
for s in lines:
    prev = None
    i = 0
    while i < len(s):
        if prev == None:
            prev = s[:n_chars_to_consider]
            i += n_chars_to_consider
            continue
        new_state = prev[1:] + s[i]
        key = prev + '->' + new_state
        if not (key in transitions):
            transitions[key] = 1
        else:
            transitions[key] += 1
        prev = new_state
        i+=1

# нормируем
summ = 0
for t in transitions.values():
    summ += t

transitions = {k: v / summ for k, v in transitions.items()}
print("Transitions count: ", len(transitions.items()))

In [ ]:
prompt_len = 11 # > n_chars_to_consider

sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

In [ ]:
# предсказание
import operator

def our_transition(pair, suf):
    k, v = pair 
    return k.startswith(suf)
        

def predict(cur_line):
    suf = cur_line[len(cur_line) - n_chars_to_consider:] 
    trs = dict()
    for t in transitions.items():
        k, v = t
        if our_transition(t, suf):
            trs[k] = v
    if len(trs) == 0:
        return None, True
    
    max_val = max(trs.values())
    max_prob_key = max(trs, key = trs.get)
    
    next_char = max_prob_key[len(max_prob_key) - 1] #last 
    return cur_line + next_char, False

print("Real sentence:\n", lines[sentence])
print('Prompt: "%s"' % prompt)
cur = prompt
for i in range(100):
    cur, ended = predict(cur)
    if ended:
        print('Достигли перехода в конец строки, всё')
        break
    print(cur)
